In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
#for installing all the libraries
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install transformers
!pip install simpletransformers
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━

In [3]:
#for checking gpu details if present any on machine
!nvidia-smi

Mon Aug  5 16:38:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
#for loading evaluate function
!pip install evaluate
!pip install tensorflow-text


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 866.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 66.2 MB/s eta 0:00:00


In [5]:
#for importing necessary all the libraries
import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm
import torch
import nltk
import evaluate

In [6]:
# to load .jsonl dataset files in the code and returning dataframe of important columns for generation task.(mainly two columns, content and their respective spoilers)
def load_dataset(file_name):
    data = []
    with open(file_name, encoding='utf8') as f:

        for line in f:
            example = json.loads(line)
            post_text = example['postText'][0]
            title = example['targetTitle']
            paragraphs = ' '.join(example['targetParagraphs'])
            spoiler = example['spoiler'][0]
            data.append({'text': post_text + ' - ' + title + paragraphs, 'spoiler': spoiler})
    return pd.DataFrame(data)
train_data = load_dataset('/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/train.jsonl')
validation_data = load_dataset('/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/val.jsonl')

In [7]:
# converting both dataframes to numpy to further apply preprocessing on it.
train_data = np.array(train_data)
validation_data = np.array(validation_data)

In [8]:
# For removing all the extra special characters from the data
characters = ['!','"','#','$','%','&','(',')','*','+','/',':',';','<','=','>','@','^','`','|','~','\t','[',']','{','}','\\','.','-']
for i in range(len(train_data)):
    for j in characters:
        train_data[i][0] = train_data[i][0].replace(j,"")
        train_data[i][1] = train_data[i][1].replace(j,"")

for i in range(len(validation_data)):
    for j in characters:
        validation_data[i][0] = validation_data[i][0].replace(j,"")
        validation_data[i][1] = validation_data[i][1].replace(j,"")


print(len(train_data))
print(len(validation_data))

train_data = pd.DataFrame(train_data)
validation_data = pd.DataFrame(validation_data)

3200
400


In [9]:
# Converting dataframes and storing them as .csv file
train_data.to_csv('/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/train.csv', index=False)
validation_data.to_csv('/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/val.csv', index=False)

In [10]:
!cd /content/drive/MyDrive/Colab Notebooks/Pegasus_data/data



/bin/bash: line 1: cd: too many arguments


In [11]:
# loading dataset in the form which is acceptable to pretrained model
from datasets import load_dataset

dataset_file = r'/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/train.csv'
dataset_file_1 = r'/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/train.csv'
train_data = load_dataset('csv', data_files=dataset_file)
val_data = load_dataset('csv', data_files=dataset_file_1)

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
print(train_data['train'][1])

{'0': "NASA sets date for full recovery of ozone hole  Hole In Ozone Layer Expected To Make Full Recovery By 2070 NASA2070 is shaping up to be a great year for Mother Earth That's when NASA scientists are predicting the hole in the ozone layer might finally make a full recovery Researchers announced their conclusion, in addition to other findings, in a presentation Wednesday during the annual American Geophysical Union meeting in San Francisco The team of scientists specifically looked at the chemical composition of the ozone hole, which has shifted in both size and depth since the passing of the Montreal Protocol in 1987 The agreement banned its 197 signatory countries from using chemicals, like chlorofluorocarbons CFCs, that break down into chlorine in the upper atmosphere and harm the ozone layer They found that, while levels of chlorine in the atmosphere have indeed decreased as a result of the protocol, it's too soon to tie them to a healthier ozone layer Ozone holes with smaller 

In [27]:
# loading pretrained model pegasus-large on gpu of the server along with it's tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model_ckpt = "microsoft/prophetnet-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

cuda


tokenizer_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


prophetnet.tokenizer:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [28]:
# Creating embeddings of the data
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['0'] , max_length = 256, truncation = True , padding= "max_length")

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['1'], max_length = 50, truncation = True , padding= "max_length")

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

train = train_data.map(convert_examples_to_features, batched = True)
val = val_data.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [29]:
print(val['train']['input_ids'])

Output hidden; open in https://colab.research.google.com to view.

In [30]:
#Intializing data collector
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [31]:
#Defining training arguments
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

trainer_args = Seq2SeqTrainingArguments(
    output_dir=r'/content/drive/MyDrive/Colab Notebooks/microsoft/prophetnet-large-uncased', num_train_epochs=10, warmup_steps=0,
    per_device_train_batch_size=2, per_device_eval_batch_size=2,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='epoch', save_steps=1e6,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:
# Passing trainer arguments
trainer = Seq2SeqTrainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=train['train'],
                  eval_dataset=val['train'])

In [33]:
# for starting training of the model
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,0.637500,0.457542
2,0.523700,0.273525
3,0.372800,0.168957
4,0.167400,0.103794
5,0.132700,0.059260
6,0.105100,0.032637
7,0.064300,0.017987
8,0.036500,0.011663
9,0.027700,0.007894
10,0.026500,0.006994


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


TrainOutput(global_step=1000, training_loss=0.4261001237332821, metrics={'train_runtime': 8993.2573, 'train_samples_per_second': 3.558, 'train_steps_per_second': 0.111, 'total_flos': 1.7646376452096e+16, 'train_loss': 0.4261001237332821, 'epoch': 10.0})

In [36]:
# For saving model after training
model_pegasus.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Pegasus_data/microsoft-prophnet-model/prophetnet_Pretrained")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


In [37]:
#For saving it's tokenizer after saving
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Pegasus_data/microsoft-prophnet-model/prophetnet_tokenizers")

('/content/drive/MyDrive/Colab Notebooks/Pegasus_data/microsoft-prophnet-model/prophetnet_tokenizers/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Pegasus_data/microsoft-prophnet-model/prophetnet_tokenizers/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Pegasus_data/microsoft-prophnet-model/prophetnet_tokenizers/prophetnet.tokenizer',
 '/content/drive/MyDrive/Colab Notebooks/Pegasus_data/microsoft-prophnet-model/prophetnet_tokenizers/added_tokens.json')

In [ ]:
import shutil
shutil.move("/content/location where ypu want to store your model", "/content/drive/MyDrive/Pegasus_data/saved_model")

'/content/drive/MyDrive/Pegasus_data/saved_model/location where ypu want to store your model'